## Preprocesamiento de datos

Aplicar operaciones sobre los datos con el fin de mejorar los modelados.

* Escalado de datos
    * StandardScaler
    * MinMaxScaler
    * RobustScaler

    Se aplican en las columnas de la X, en columnas numéricas

In [1]:
import seaborn as sns 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer, PowerTransformer, OneHotEncoder, LabelEncoder, KBinsDiscretizer, Binarizer
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error, accuracy_score


In [2]:
df = sns.load_dataset('diamonds').dropna().sample(5000, random_state=42).reset_index(drop=True) # muestra para hacer pruebas
#df = sns.load_dataset('diamonds').dropna().reset_index(drop=True) # muestra para hacer pruebas
df.head(3)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.24,Ideal,G,VVS1,62.1,56.0,559,3.97,4.00,2.47
1,0.58,Very Good,F,VVS2,60.0,57.0,2201,5.44,5.42,3.26
2,0.40,Ideal,E,VVS2,62.1,55.0,1238,4.76,4.74,2.95


In [3]:


# Particionar y crear método calculate_metrics para hacer un modelado antes de hacer nada y ver si aplicando preprocesadores mejora
X = df[['carat', 'depth', 'table', 'x', 'y', 'z']]
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

df_resultados = pd.DataFrame(columns=['Modelo', 'Preprocesado', 'R2', 'MAE', 'RMSE', 'MAPE'])

def calculate_metrics(preprocessor_name, X_train, X_test, y_train, y_test):
    models = {
        'LinearRegression': LinearRegression(),
        'KNN': KNeighborsRegressor(),
        'SVR': SVR(),
        'DecisionTree': DecisionTreeRegressor(random_state=42),
        'RandomForest': RandomForestRegressor(random_state=42)
    }
    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        df_resultados.loc[len(df_resultados)] = [model_name, preprocessor_name, r2_score(y_test, y_pred), mean_absolute_error(y_test, y_pred),root_mean_squared_error(y_test, y_pred),mean_absolute_percentage_error(y_test, y_pred)]
    
    return df_resultados.sort_values('R2', ascending=False)

In [4]:
calculate_metrics('Sin preprocesado', X_train, X_test, y_train, y_test)

,Modelo,Preprocesado,R2,MAE,RMSE,MAPE
4,RandomForest,Sin preprocesado,0.867955,835.518392,1500.214999,0.210188
0,LinearRegression,Sin preprocesado,0.861482,930.832368,1536.548541,0.288899
1,KNN,Sin preprocesado,0.849006,899.186200,1604.254748,0.226420
3,DecisionTree,Sin preprocesado,0.766569,1122.743000,1994.676075,0.274400
2,SVR,Sin preprocesado,-0.179999,2967.146616,4484.706331,1.100950


## StandardScaler

Transforma los datos para que cada característica tenga media 0 y desviación estándar 1.  

$$
X_{\text{scaled}} = \frac{X - \mu}{\sigma}
$$

donde $\mu$ es la media y $\sigma$ la desviación estándar (calculados **solamente** en el conjunto de entrenamiento).
  
**Cuándo usarlo**:  
- Cuando los datos no tienen outliers extremadamente grandes (o son relativamente cercanos a una distribución normal).  
- Es el escalado más común, especialmente para algoritmos que asumen normalidad o que son sensibles a la escala (regresiones lineales, redes neuronales, SVM, etc.).

In [5]:
scaler = StandardScaler()
scaler.fit(X_train) # fit solo sobre train y no en test para evitar data leakage

X_train_scaled = scaler.transform(X_train) # devuelve un array de numpy
X_test_scaled = scaler.transform(X_test) # devuelve un array de numpy

# opcional, pasarlo a dataframes de pandas con los nombres de las columnas
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)
X_train_scaled.head(2)

,carat,depth,table,x,y,z
0,-1.028331,0.973355,-0.630227,-1.277261,-1.259030,-1.173678
1,0.439665,2.353854,0.692667,0.507249,0.500558,0.810866


In [6]:
calculate_metrics('StandardScaler', X_train_scaled, X_test_scaled, y_train, y_test)

,Modelo,Preprocesado,R2,MAE,RMSE,MAPE
9,RandomForest,StandardScaler,0.868495,834.839074,1497.149208,0.210013
4,RandomForest,Sin preprocesado,0.867955,835.518392,1500.214999,0.210188
5,LinearRegression,StandardScaler,0.861482,930.832368,1536.548541,0.288899
0,LinearRegression,Sin preprocesado,0.861482,930.832368,1536.548541,0.288899
6,KNN,StandardScaler,0.859353,874.932000,1548.314323,0.221851
1,KNN,Sin preprocesado,0.849006,899.186200,1604.254748,0.226420
8,DecisionTree,StandardScaler,0.766985,1119.155000,1992.897756,0.273665
3,DecisionTree,Sin preprocesado,0.766569,1122.743000,1994.676075,0.274400
7,SVR,StandardScaler,0.033553,2363.397416,4058.655409,0.675242
2,SVR,Sin preprocesado,-0.179999,2967.146616,4484.706331,1.100950


## Minmaxscaler
Escala y traslada cada característica individual a un rango definido, por defecto $[0,1]$.  

$$
X_{\text{scaled}} = \frac{X - X_{\min}}{X_{\max} - X_{\min}}
$$


**Cuándo usarlo**:

- Cuando quieres que los datos estén **acotados entre 0 y 1** o entre otro rango definido (por ejemplo, $[-1, 1]$), porque se puede personalizar el rango a $[min, max]$. Para algoritmos basados en distancias como KNN.
- Sin embargo, **es muy sensible a los outliers**. Un valor muy grande puede comprimir el resto de datos.

In [7]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train) # se puede hacer el fit y el transform aparte o se puede hacer todo en fit_transform()
X_test_scaled = scaler.transform(X_test) # devuelve un array de numpy

# opcional, pasarlo a dataframes de pandas con los nombres de las columnas
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)
X_train_scaled.head(2)

,carat,depth,table,x,y,z
0,0.026247,0.500,0.291667,0.066773,0.077901,0.265306
1,0.209974,0.625,0.416667,0.386328,0.391097,0.546939


In [8]:
calculate_metrics('MinMaxScaler', X_train_scaled, X_test_scaled, y_train, y_test)

,Modelo,Preprocesado,R2,MAE,RMSE,MAPE
14,RandomForest,MinMaxScaler,0.868646,835.654962,1496.286474,0.210257
9,RandomForest,StandardScaler,0.868495,834.839074,1497.149208,0.210013
4,RandomForest,Sin preprocesado,0.867955,835.518392,1500.214999,0.210188
5,LinearRegression,StandardScaler,0.861482,930.832368,1536.548541,0.288899
0,LinearRegression,Sin preprocesado,0.861482,930.832368,1536.548541,0.288899
10,LinearRegression,MinMaxScaler,0.861482,930.832368,1536.548541,0.288899
6,KNN,StandardScaler,0.859353,874.932000,1548.314323,0.221851
11,KNN,MinMaxScaler,0.857112,868.646400,1560.600343,0.220208
1,KNN,Sin preprocesado,0.849006,899.186200,1604.254748,0.226420
8,DecisionTree,StandardScaler,0.766985,1119.155000,1992.897756,0.273665


## Robustscaler
Escala los datos usando **mediana** e **IQR** (rango intercuartílico).  

$$
X_{\text{scaled}} = \frac{X - \text{mediana}(X)}{\text{IQR}}
$$
donde $\text{IQR} = Q_3 - Q_1$.

**Cuándo usarlo**:

Cuando existen **outliers** en los datos que pueden afectar mucho al escalado.
Al usar la mediana y el IQR en lugar de la media y desviación estándar, resulta mucho **menos sensible a valores atípicos**.

In [9]:
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test) 

# opcional, pasarlo a dataframes de pandas con los nombres de las columnas
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)
X_train_scaled.head(2)

,carat,depth,table,x,y,z
0,-0.62500,0.866667,-0.333333,-0.768176,-0.767956,-0.707965
1,0.46875,2.200000,0.666667,0.334705,0.320442,0.513274


In [10]:
calculate_metrics('RobustScaler', X_train_scaled, X_test_scaled, y_train, y_test)

,Modelo,Preprocesado,R2,MAE,RMSE,MAPE
14,RandomForest,MinMaxScaler,0.868646,835.654962,1496.286474,0.210257
9,RandomForest,StandardScaler,0.868495,834.839074,1497.149208,0.210013
19,RandomForest,RobustScaler,0.868256,835.223960,1498.505051,0.210161
4,RandomForest,Sin preprocesado,0.867955,835.518392,1500.214999,0.210188
5,LinearRegression,StandardScaler,0.861482,930.832368,1536.548541,0.288899
0,LinearRegression,Sin preprocesado,0.861482,930.832368,1536.548541,0.288899
15,LinearRegression,RobustScaler,0.861482,930.832368,1536.548541,0.288899
10,LinearRegression,MinMaxScaler,0.861482,930.832368,1536.548541,0.288899
6,KNN,StandardScaler,0.859353,874.932000,1548.314323,0.221851
16,KNN,RobustScaler,0.858300,876.919000,1554.096430,0.221305


## QuantileTransformer

Es una transformación basada en **cuantiles**:  

1. Ordena los valores de cada columna y les asigna su posición cuantílica (e.g. percentiles).  

2. Mapea esos cuantiles ya sea a una distribución **uniforme** en $[0,1]$ o a una distribución **normal** (Gaussiana) si se especifica `output_distribution='normal'`.

- Por defecto, `output_distribution='uniform'`, lo que hace que cada característica se distribuya aproximadamente **de manera uniforme** en $[0, 1]$.  

- Si pones `output_distribution='normal'`, intentará que los datos se parezcan a una **distribución normal (Gaussiana)** con media 0 y desviación estándar 1.

¿Cuándo usarlo?

- Cuando quieres aplanar la distribución de una variable que está muy sesgada (skewed) o con colas largas. El método de cuantiles “estira” y “comprime” la distribución de forma que cada cuantil se mapea a un cuantil de la distribución objetivo (uniforme o normal).

- Es útil cuando quieres datos:
  - Bien distribuidos entre $[0, 1]$ (caso uniforme).
  - O aproximar una Gaussiana sin realizar transformaciones paramétricas (e.g. logaritmo).

Puede ser más fuerte que raíz o logaritmo porque no solo reduce sesgo redistribuye los valores, puede ser demasiado agresivo si los datos ya son simétricos.

In [11]:
X_train.skew()

carat    1.193090
depth    0.159501
table    0.681555
x        0.434562
y        0.434972
z        0.407144
dtype: float64

In [12]:
transformed = QuantileTransformer()
X_train_transformed = transformed.fit_transform(X_train)
X_test_transformed = transformed.transform(X_test) 
X_train_transformed = pd.DataFrame(X_train_transformed, columns=X.columns)
X_test_transformed = pd.DataFrame(X_test_transformed, columns=X.columns)
X_train_transformed.head(2)

,carat,depth,table,x,y,z
0,0.059560,0.883383,0.285285,0.041041,0.057558,0.131632
1,0.670671,0.983984,0.773273,0.657157,0.657157,0.787788


In [13]:
X_train_transformed.skew()

carat    0.001497
depth   -0.000490
table    0.007868
x       -0.000119
y        0.000009
z        0.000117
dtype: float64

In [14]:
calculate_metrics('QuantileTransformer', X_train_transformed, X_test_transformed, y_train, y_test)

,Modelo,Preprocesado,R2,MAE,RMSE,MAPE
14,RandomForest,MinMaxScaler,0.868646,835.654962,1496.286474,0.210257
9,RandomForest,StandardScaler,0.868495,834.839074,1497.149208,0.210013
24,RandomForest,QuantileTransformer,0.868345,835.537068,1498.000096,0.210055
19,RandomForest,RobustScaler,0.868256,835.223960,1498.505051,0.210161
4,RandomForest,Sin preprocesado,0.867955,835.518392,1500.214999,0.210188
0,LinearRegression,Sin preprocesado,0.861482,930.832368,1536.548541,0.288899
5,LinearRegression,StandardScaler,0.861482,930.832368,1536.548541,0.288899
15,LinearRegression,RobustScaler,0.861482,930.832368,1536.548541,0.288899
10,LinearRegression,MinMaxScaler,0.861482,930.832368,1536.548541,0.288899
6,KNN,StandardScaler,0.859353,874.932000,1548.314323,0.221851


## PowerTransformer

intenta que los datos sean parecidos a una distribución normal.

PowerTransformer aplica transformaciones de **potencia** para hacer que los datos se acerquen más a una distribución normal.

- Admite dos métodos principales:

  1. **Box-Cox**: requiere que todos los datos sean **estrictamente positivos**.  
  2. **Yeo-Johnson**: puede manejar datos con valores 0 o negativos.  Yeo-Johnson es una versión mejorada de Box-Cox que funciona con valores negativos y positivos.

Internamente, `PowerTransformer` encuentra el mejor parámetro de potencia que estabiliza la varianza y reduce la asimetría (skew) de los datos, por tanto es una opción más flexible y automatizada que aplicar manualmente un np.sqrt o np.log a una columna.

¿Cuándo usarlo?

- Cuando tus datos están fuertemente sesgados (tienen heavy skew) y necesitas **mejorar la normalidad**. El QuantileTransfomer podría ser más fuerte. 

- Se suele usar antes de **modelos lineales** o algoritmos que asumen distribuciones aproximadamente gaussianas, ayudando a cumplir hipótesis de homocedasticidad (misma varianza) y mejorando la linealidad.  

- Si tus datos tienen valores cero o negativos, no puedes usar Box-Cox, pero sí Yeo-Johnson.
- Puedes luego aplicar un escalado adicional (por ejemplo, `StandardScaler`) tras la transformación de potencia si lo deseas.

In [15]:
transformed = PowerTransformer()
X_train_transformed = transformed.fit_transform(X_train)
X_test_transformed = transformed.transform(X_test) 
X_train_transformed = pd.DataFrame(X_train_transformed, columns=X.columns)
X_test_transformed = pd.DataFrame(X_test_transformed, columns=X.columns)
X_train_transformed.head(2)

,carat,depth,table,x,y,z
0,-1.332196,0.974480,-0.592305,-1.430225,-1.403233,-1.264652
1,0.715544,2.309622,0.764271,0.613714,0.607868,0.866351


In [16]:
calculate_metrics('PowerTransformer', X_train_transformed, X_test_transformed, y_train, y_test)

,Modelo,Preprocesado,R2,MAE,RMSE,MAPE
14,RandomForest,MinMaxScaler,0.868646,835.654962,1496.286474,0.210257
9,RandomForest,StandardScaler,0.868495,834.839074,1497.149208,0.210013
24,RandomForest,QuantileTransformer,0.868345,835.537068,1498.000096,0.210055
19,RandomForest,RobustScaler,0.868256,835.223960,1498.505051,0.210161
29,RandomForest,PowerTransformer,0.868020,834.399099,1499.846193,0.209508
4,RandomForest,Sin preprocesado,0.867955,835.518392,1500.214999,0.210188
5,LinearRegression,StandardScaler,0.861482,930.832368,1536.548541,0.288899
0,LinearRegression,Sin preprocesado,0.861482,930.832368,1536.548541,0.288899
15,LinearRegression,RobustScaler,0.861482,930.832368,1536.548541,0.288899
10,LinearRegression,MinMaxScaler,0.861482,930.832368,1536.548541,0.288899


In [17]:
print('skew antes: \n', X_train.skew())
print('skew después: \n', X_train_transformed.skew())

skew antes: 
 carat    1.193090
depth    0.159501
table    0.681555
x        0.434562
y        0.434972
z        0.407144
dtype: float64
skew después: 
 carat    0.127325
depth    0.002243
table   -0.005639
x        0.037489
y        0.037876
z        0.030497
dtype: float64


In [18]:
transformed = PowerTransformer(standardize=False)
X_train_transformed = transformed.fit_transform(X_train)
X_test_transformed = transformed.transform(X_test) 

calculate_metrics('PowerTransformer standFalse', X_train_transformed, X_test_transformed, y_train, y_test)

,Modelo,Preprocesado,R2,MAE,RMSE,MAPE
14,RandomForest,MinMaxScaler,0.868646,835.654962,1496.286474,0.210257
9,RandomForest,StandardScaler,0.868495,834.839074,1497.149208,0.210013
24,RandomForest,QuantileTransformer,0.868345,835.537068,1498.000096,0.210055
19,RandomForest,RobustScaler,0.868256,835.223960,1498.505051,0.210161
29,RandomForest,PowerTransformer,0.868020,834.399099,1499.846193,0.209508
4,RandomForest,Sin preprocesado,0.867955,835.518392,1500.214999,0.210188
34,RandomForest,PowerTransformer standFalse,0.863769,842.218551,1523.808319,0.211880
0,LinearRegression,Sin preprocesado,0.861482,930.832368,1536.548541,0.288899
5,LinearRegression,StandardScaler,0.861482,930.832368,1536.548541,0.288899
15,LinearRegression,RobustScaler,0.861482,930.832368,1536.548541,0.288899


## OneHotEncoder

Equivale a pd.get_dummies de pandas pero es de Scikit Learn.

Crea **columnas binarias (dummies)** para cada categoría de una **feature** nominal.

**Cuándo usarlo**:  

1. Para **features categóricas nominales** (sin orden), como color, ciudad, tipo de mascota, etc.  
2. Normalmente se aplica a **variables de entrada** (X).  
3. Útil en la mayoría de los modelos que necesitan variables numéricas y no tienen forma de manejar directamente categorías.
4. Se puede usar en pipelines de scikit learn

Parámetro sparse_output:

* sparse_output=True: Devuelve la transformación como una matriz dispersa (scipy.sparse.csr_matrix) en lugar de un numpy.ndarray. 
    * Ventaja: Usa menos memoria si hay muchas categorías con muchos ceros (matriz dispersa)
    * Desventaja: Puede ser incompatible con algunas funciones de Pandas y Scikit-learn que esperan una matriz densa.
* sparse_output=False: Devuelve la transformación como un array denso (numpy.ndarray), en lugar de una matriz dispersa.
    * Ventaja: Se puede convertir fácilmente en un DataFrame de Pandas sin errores ni conversiones adicionales.
    * Desventaja: Puede consumir más memoria si hay muchas categorías y muchos ceros.

* Diferencia entre matriz densa y dispersa:
    * Matriz densa: Es una matriz donde todos los valores, incluyendo los ceros, son almacenados en memoria.
    * Matriz dispersa: Es una matriz en la que se almacenan solo los valores distintos de cero, junto con sus coordenadas (índices de fila y columna). Más óptima pero más difícil de manipular directamente, requiere conversión a formato denso para ciertas operaciones.

In [19]:
X = df[['carat', 'depth', 'table', 'x', 'y', 'z', 'cut', 'color', 'clarity']]
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)



In [20]:
#obtener nombres de columnas numéricas y categóricas
numerical_columns = X_train.select_dtypes(exclude=['object', 'category']).columns.to_list()
categorical_columns = X_train.select_dtypes(include=['object', 'category']).columns.to_list()

encoder = OneHotEncoder(sparse_output=False) #separate_output=False para obtener la matriz de 0 y 1 
X_train_encoded = encoder.fit_transform(X_train[categorical_columns]) #array de numpy con las codificaciones
X_test_encoded = encoder.transform(X_test[categorical_columns]) 

# pasarlo a dataframe de pandas y juntarlo con las numericas para obtener resultado como pd.get_dummies
X_train_final = pd.concat(
    [
        pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out()).reset_index(drop=True),
        X_train[numerical_columns].reset_index(drop=True)
    ], 
    axis=1
)

X_test_final = pd.concat(
    [
        pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out()).reset_index(drop= True), 
        X_test[numerical_columns].reset_index(drop=True)
    ],
    axis=1
)
X_test_final.head()

,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,color_H,...,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,carat,depth,table,x,y,z
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.70,60.6,58.0,5.80,5.72,3.49
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.03,61.0,60.0,6.46,6.53,3.96
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.31,62.6,57.0,4.33,4.29,2.70
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.00,62.7,58.0,6.41,6.32,3.99
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.50,61.6,55.0,5.11,5.14,3.16


In [21]:
calculate_metrics('OneHotEncoder', X_train_final, X_test_final, y_train, y_test)

,Modelo,Preprocesado,R2,MAE,RMSE,MAPE
39,RandomForest,OneHotEncoder,0.963541,397.252065,788.302258,0.097110
38,DecisionTree,OneHotEncoder,0.930697,528.805000,1086.851350,0.126546
35,LinearRegression,OneHotEncoder,0.914045,791.194142,1210.402235,0.438605
36,KNN,OneHotEncoder,0.887188,764.857600,1386.662491,0.201587
14,RandomForest,MinMaxScaler,0.868646,835.654962,1496.286474,0.210257
9,RandomForest,StandardScaler,0.868495,834.839074,1497.149208,0.210013
24,RandomForest,QuantileTransformer,0.868345,835.537068,1498.000096,0.210055
19,RandomForest,RobustScaler,0.868256,835.223960,1498.505051,0.210161
29,RandomForest,PowerTransformer,0.868020,834.399099,1499.846193,0.209508
4,RandomForest,Sin preprocesado,0.867955,835.518392,1500.214999,0.210188


In [22]:
# Combinar OneHotEncoder con MinMaxScaler
# obtener nombres de columnas numéricas y categóricas
numerical_columns = X_train.select_dtypes(exclude=['object', 'category']).columns.to_list() # np.number alternativa
categorical_columns = X_train.select_dtypes(include=['object', 'category']).columns.to_list()

encoder = OneHotEncoder(sparse_output=False) # sparse_output=False para obtenerlo como matriz de 0s y 1s , probar drop='first'
X_train_encoded = encoder.fit_transform(X_train[categorical_columns]) # array de numpy con las codificaciones
X_test_encoded = encoder.transform(X_test[categorical_columns])

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train[numerical_columns])
X_test_scaled = scaler.transform(X_test[numerical_columns])

X_train_final = pd.concat(
    [
        pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out()).reset_index(drop=True), # categoricas
        pd.DataFrame(X_train_scaled, columns=numerical_columns).reset_index(drop=True) # numéricas
    ],
    axis=1
)
X_test_final = pd.concat(
    [
        pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out()).reset_index(drop=True), # categoricas
        pd.DataFrame(X_test_scaled, columns=numerical_columns).reset_index(drop=True) # numéricas
    ],
    axis=1
)
X_test_final.head()

,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,color_H,...,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,carat,depth,table,x,y,z
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.131234,0.33750,0.375000,0.310016,0.303657,0.424490
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.217848,0.36250,0.458333,0.414944,0.432432,0.520408
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.028871,0.46250,0.333333,0.076312,0.076312,0.263265
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.209974,0.46875,0.375000,0.406995,0.399046,0.526531
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.078740,0.40000,0.250000,0.200318,0.211447,0.357143


In [23]:
calculate_metrics('OneHotEncoder+MinMaxScaler', X_train_final, X_test_final, y_train, y_test)

,Modelo,Preprocesado,R2,MAE,RMSE,MAPE
39,RandomForest,OneHotEncoder,0.963541,397.252065,788.302258,0.097110
44,RandomForest,OneHotEncoder+MinMaxScaler,0.963456,397.241947,789.227699,0.097156
43,DecisionTree,OneHotEncoder+MinMaxScaler,0.935301,519.908000,1050.125344,0.126376
38,DecisionTree,OneHotEncoder,0.930697,528.805000,1086.851350,0.126546
40,LinearRegression,OneHotEncoder+MinMaxScaler,0.914045,791.194142,1210.402235,0.438605
35,LinearRegression,OneHotEncoder,0.914045,791.194142,1210.402235,0.438605
36,KNN,OneHotEncoder,0.887188,764.857600,1386.662491,0.201587
14,RandomForest,MinMaxScaler,0.868646,835.654962,1496.286474,0.210257
9,RandomForest,StandardScaler,0.868495,834.839074,1497.149208,0.210013
24,RandomForest,QuantileTransformer,0.868345,835.537068,1498.000096,0.210055


## LabelEncoder

Convierte etiquetas (categorías) a valores numéricos enteros de 0 a n-1.  

Por ejemplo, si tienes las categorías `["rojo", "verde", "azul"]`, podría asignar  
- *rojo* $\to 0$,  
- *verde* $\to 1$,  
- *azul* $\to 2$.

* Normalmente se usa para la variable de salida (y) si se trata de un problema de clasificación multiclase.
* Convierte cada clase categórica a un entero distinto.
* También puede usarse en columnas de entrada si (y solo si) tienen un orden real (caso ordinal) o si el modelo puede manejarlo sin suponer que 2 > 1 > 0 (pero esto no es común; en features categóricas nominales, lo típico es OneHotEncoder).

Equivalente a cuando hacemos el `df['class'].map({'setosa':0, 'virginica':1, 'versicolor':2})`

In [24]:
X = df[['carat', 'depth', 'table', 'x', 'y', 'z', 'price']]
y = df['cut']

In [25]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.20, random_state=42)
print('clases del encoder: ', encoder.classes_)
print('ejemplo y_encoded: ', y_encoded[:10])
print('ejemplo y_train: ', y_train[:10])
print('ejemplo y_test: ', y_test[:10])

clases del encoder:  ['Fair' 'Good' 'Ideal' 'Premium' 'Very Good']
ejemplo y_encoded:  [2 4 2 3 2 0 2 2 3 2]
ejemplo y_train:  [1 0 3 4 1 2 2 2 2 4]
ejemplo y_test:  [3 3 2 3 2 2 2 2 2 1]


### inverse_transform()

In [26]:
#con inverse_transform podemos obtener las categorias originales a partir de los datos codificados
# podemos aplicar inverse_transform sobre y_pred para obtener las categorias de las predicciones si queremos
encoder.inverse_transform(y_encoded)

array(['Ideal', 'Very Good', 'Ideal', ..., 'Ideal', 'Very Good',
       'Very Good'], shape=(5000,), dtype=object)

In [27]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)


0.749

In [28]:
print('predicciones', y_pred[:10])
print('predicciones decodificadas', encoder.inverse_transform(y_pred)[:10])

predicciones [3 3 2 3 2 2 2 2 2 1]
predicciones decodificadas ['Premium' 'Premium' 'Ideal' 'Premium' 'Ideal' 'Ideal' 'Ideal' 'Ideal'
 'Ideal' 'Good']


## KBinsdiscretizer

Similar a pd.cut de pandas para discretizar columnas numéricas.

Convierte variables numéricas continuas en variables discretas, dividiendo los valores en **intervalos o "bins"**. Cada intervalo recibe una etiqueta numérica.

¿Cuándo usarlo?

* Cuando queremos convertir variables numéricas continuas en categorías discretas (por ejemplo, dividir `carat` en "pequeño", "mediano" y "grande").  
* Cuando un modelo puede beneficiarse de información categorizada en lugar de valores continuos.  
* Para mejorar la interpretabilidad de un modelo.  

Formas de discretización:

- `uniform`: Divide el rango en intervalos de **igual tamaño**.
- `quantile`: Crea intervalos con **igual número de muestras**.
- `kmeans`: Usa **K-Means** para definir los bins.

In [29]:
# realizar clasificacion multiclase sobre price
X = df[['carat', 'depth', 'table', 'x', 'y', 'z']]
y = df[['price']] # [[]] para que sea 2d para discretizar, col numerica que transformaremos en categorica --> clasificacion multiclase

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


In [30]:
# discretizar precio en 4 grupos
discretizer = KBinsDiscretizer(encode='ordinal', n_bins=4, strategy='kmeans') #encode='onehot-dense' genera matriz densa estilo one hot para la x
discretizer.fit(y_train)

y_train_dicretized = discretizer.transform(y_train).ravel() #convierte de 2d a 1d para usar en scikit fit y predict
y_test_dicretized = discretizer.transform(y_test).ravel()

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train_dicretized)
y_pred = model.predict(X_test)
accuracy_score(y_test_dicretized, y_pred)



0.853

In [31]:
print('predicciones', y_pred[:10])


predicciones [0. 1. 0. 1. 0. 0. 0. 1. 1. 1.]


In [32]:
print('discretizer.n_bins_:', discretizer.n_bins_)
print('discretizer.n_features_in_:', discretizer.n_features_in_)
print('discretizer.n_features_in_:', discretizer.feature_names_in_)
print('discretizer.bin_edges_:', discretizer.bin_edges_)
print('bin min', discretizer.bin_edges_[0][0])
print('bin 1', discretizer.bin_edges_[0][1])
print('bin 2', discretizer.bin_edges_[0][2])
print('bin 3', discretizer.bin_edges_[0][3])
print('bin max', discretizer.bin_edges_[0][4])

discretizer.n_bins_: [4]
discretizer.n_features_in_: 1
discretizer.n_features_in_: ['price']
discretizer.bin_edges_: [array([  336.        ,  2964.95252869,  6925.92917442, 12188.07840494,
        18823.        ])                                               ]
bin min 336.0
bin 1 2964.952528685514
bin 2 6925.929174424704
bin 3 12188.078404935182
bin max 18823.0


## Binarizer
Binarizer convierte valores numéricos en valores binarios (0 o 1) en función de un umbral. 

Se usa cuando quieres transformar una variable numérica en categórica, lo que puede ser útil para mejorar el rendimiento de algunos modelos.

Por ejemplo podemos convertir la variable precio a una variable binaria barato (0) y caro (1) para realizar clasificación binaria.

Otro ejemplo es binarizar la edad de una persona en adulto (0 o 1) en función de si tiene igual o más de 18 años o no.

In [33]:
X = df[['carat', 'depth', 'table', 'x', 'y', 'z']]
y = df[['price']] # [[]] para que sea 2d para discretizar, col numerica que transformaremos en categorica --> clasificacion multiclase

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

binarizer = Binarizer(threshold=df['carat'].median())
X_train['carat'] = binarizer.fit_transform(X_train[['carat']])
X_test['carat'] = binarizer.transform(X_test[['carat']])
X_train.head(3)

,carat,depth,table,x,y,z
4227,0.0,63.2,56.0,4.27,4.30,2.71
4676,1.0,65.2,59.0,6.28,6.27,4.09
800,1.0,61.3,58.0,6.10,6.04,3.72
